<a href="https://colab.research.google.com/github/LudovicSaintBauzel/AI4Health/blob/main/SummerSchool2023_IA_Sante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialisation des librairies pour le travail.

In [1]:
import pandas as pd
import os
import glob
import math
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Chargement des données

In [32]:
!curl -O https://nuage.isir.upmc.fr/index.php/s/ZiPJ3RFGFaF3HBs/download/records.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18.9M  100 18.9M    0     0  10.7M      0  0:00:01  0:00:01 --:--:-- 10.7M


# Analyse et travail sur les données

In [35]:
!unzip -o records.zip
!ls 

Archive:  records.zip
  inflating: records/.DS_Store       
  inflating: __MACOSX/records/._.DS_Store  
  inflating: records/DESCRIPTIONS_DONNEES.txt  
  inflating: __MACOSX/records/._DESCRIPTIONS_DONNEES.txt  
 extracting: records/move/testen.txt  
  inflating: records/move/follow_traj.txt  
  inflating: records/move/fuzzyout.txt  
 extracting: records/move/traj.txt   
  inflating: records/move/trajout.txt  
 extracting: records/move/rawsinal.txt  
  inflating: records/move/move_effort.txt  
 extracting: records/OL/testen.txt   
  inflating: records/OL/follow_traj.txt  
  inflating: records/OL/fuzzyout.txt  
 extracting: records/OL/traj.txt     
  inflating: records/OL/trajout.txt  
 extracting: records/OL/rawsinal.txt  
 extracting: records/OL/move_effort.txt  
  inflating: records/fx/.DS_Store    
  inflating: __MACOSX/records/fx/._.DS_Store  
  inflating: records/fuzzy/.DS_Store  
  inflating: __MACOSX/records/fuzzy/._.DS_Store  
 extracting: records/MMESA/testen.txt  
  inflating:

## Observons l'organisation des données

Maintenant que les données sont téléchargées, il est possible de connaître l'organisation des données en ouvrant le fichier (/content/records/DESCRIPTIONS_DONNEES.txt). On note que le fichier follow_traj.txt contient toutes les informations utiles pour la classification que nous souhaitons effectuer. 


In [36]:
s = [[file for file in glob.glob("*/*/*/follow_traj.txt")]]

flats = []
for sub_s in s:
    for item in sub_s:
        flats.append(item)
print(flats)


['records/BO2504/test/follow_traj.txt', 'records/MA/traj1/follow_traj.txt', 'records/MA/move4/follow_traj.txt', 'records/MA/traj7/follow_traj.txt', 'records/MA/traj6/follow_traj.txt', 'records/MA/move2/follow_traj.txt', 'records/MA/traj10/follow_traj.txt', 'records/MA/traj2/follow_traj.txt', 'records/MA/fx10/follow_traj.txt', 'records/MA/fx3/follow_traj.txt', 'records/MA/traj4/follow_traj.txt', 'records/MA/fuz3/follow_traj.txt', 'records/MA/move5/follow_traj.txt', 'records/MA/fx6/follow_traj.txt', 'records/MA/fx8/follow_traj.txt', 'records/MA/move3/follow_traj.txt', 'records/MA/fuz2/follow_traj.txt', 'records/MA/move7/follow_traj.txt', 'records/MA/traj3/follow_traj.txt', 'records/MA/fx4/follow_traj.txt', 'records/MA/fuz4/follow_traj.txt', 'records/MA/move6/follow_traj.txt', 'records/MA/fx2/follow_traj.txt', 'records/MA/traj5/follow_traj.txt', 'records/MA/traj9/follow_traj.txt', 'records/MA/fuz1/follow_traj.txt', 'records/MA/traj8/follow_traj.txt', 'records/MA/fx7/follow_traj.txt', 'rec

In [51]:
df = pd.read_csv(flats[0], sep='\s+', header=None)
df['file'] = pd.Series([flats[1] for i in df[0]])
print(df)

EmptyDataError: ignored

In [48]:
for f in flats:
  fd_stat = os.stat(f)
  if fd_stat.st_size==0:
    print('Fichier à retirer : ' + f + " car fichier vide")

Fichier à retirer : records/BO2504/test/follow_traj.txt car fichier vide
Fichier à retirer : records/fuzzy/tt/follow_traj.txt car fichier vide
Fichier à retirer : records/test1/traj/follow_traj.txt car fichier vide


In [52]:
df = pd.read_csv(flats[1], sep='\s+', header=None)
df['file'] = pd.Series([flats[1] for i in df[0]])
print(df)

        0            1           2          3          4         5         6  \
0   -1595  1058.074518  761.656680   0.000000  -1.037723  2.066674  4.775655   
1   -1556  1058.074518  761.656680   0.000000  -0.215120  1.761053  4.041022   
2   -1543  1055.289818  765.055445   0.000000  -0.454037  1.366076  3.088639   
3   -1530  1058.074518  761.656680   0.361185  -1.829286  0.882182  1.928515   
4   -1517  1058.074518  761.656680   1.273799  -4.176914  0.323786  0.629320   
..    ...          ...         ...        ...        ...       ...       ...   
442  3890   838.417968  984.463406 -83.621313 -15.566355 -5.002669  0.385678   
443  3903   838.417968  984.463406 -83.621313 -15.517402 -5.053957  0.205262   
444  3916   838.417968  984.463406 -83.621313 -15.563012 -5.137251 -0.006967   
445  3929   838.417968  984.463406 -83.621313 -15.737026 -5.253628 -0.231495   
446  3935   838.417968  984.463406 -83.621313 -16.049570 -5.392206 -0.438137   

             7         8         9  ...

In [64]:
df[19]

0     -0.0
1     -0.0
2     -0.0
3     -0.0
4     -0.0
      ... 
442   -0.0
443   -0.0
444   -0.0
445   -0.0
446   -0.0
Name: 19, Length: 447, dtype: float64

In [67]:
filtering_data = ['records/test1/traj/follow_traj.txt',
                  'records/BO2504/test/follow_traj.txt',
                  'records/fuzzy/tt/follow_traj.txt',
                  'records/test1/fx/follow_traj.txt',
                  # Above Problematic data : Below test data not relevant
                  'records/test1/av/follow_traj.txt',
                  'records/test2/traj/follow_traj.txt',
                  'records/test2/av/follow_traj.txt',
                  'records/traj/test/follow_traj.txt',
                  'records/move/follow_traj.txt',
                  'NO_march1-3/follow_traj.txt',
                  'NO_march6-10/follow_traj.txt',
                  'GU_marche1/follow_traj.txt',
                  'GU_marche2/follow_traj.txt'
                  ]

for ff in flats[1:]:
    dftmp = []
    if ff in filtering_data:
        print("File filtered not loaded : "+str(ff)+"\n")
        continue
    print("File opened and loaded : " + str(ff) + "\n")
    dftmp = pd.read_csv(ff, sep='\s+', header=None)
    if len(dftmp) > 0:
        print("Appending len(dftmp):" + str(len(dftmp))+"\n")
        dftmp['file'] = pd.Series([ff for i in dftmp[0]])
        df = df.append(dftmp)

print('len(df)=' + str(len(df)))

File opened and loaded : records/MA/traj1/follow_traj.txt

Appending len(dftmp):447

File opened and loaded : records/MA/move4/follow_traj.txt

Appending len(dftmp):1114

File opened and loaded : records/MA/traj7/follow_traj.txt

Appending len(dftmp):411

File opened and loaded : records/MA/traj6/follow_traj.txt

Appending len(dftmp):336

File opened and loaded : records/MA/move2/follow_traj.txt

Appending len(dftmp):1039

File opened and loaded : records/MA/traj10/follow_traj.txt

Appending len(dftmp):353

File opened and loaded : records/MA/traj2/follow_traj.txt

Appending len(dftmp):384

File opened and loaded : records/MA/fx10/follow_traj.txt

Appending len(dftmp):958

File opened and loaded : records/MA/fx3/follow_traj.txt

Appending len(dftmp):442

File opened and loaded : records/MA/traj4/follow_traj.txt

Appending len(dftmp):494

File opened and loaded : records/MA/fuz3/follow_traj.txt

Appending len(dftmp):1225

File opened and loaded : records/MA/move5/follow_traj.txt



<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27:

Appending len(dftmp):1130

File opened and loaded : records/MA/fx6/follow_traj.txt

Appending len(dftmp):1463

File opened and loaded : records/MA/fx8/follow_traj.txt

Appending len(dftmp):951

File opened and loaded : records/MA/move3/follow_traj.txt

Appending len(dftmp):1125

File opened and loaded : records/MA/fuz2/follow_traj.txt

Appending len(dftmp):420

File opened and loaded : records/MA/move7/follow_traj.txt

Appending len(dftmp):1118

File opened and loaded : records/MA/traj3/follow_traj.txt

Appending len(dftmp):400

File opened and loaded : records/MA/fx4/follow_traj.txt

Appending len(dftmp):872

File opened and loaded : records/MA/fuz4/follow_traj.txt

Appending len(dftmp):1313

File opened and loaded : records/MA/move6/follow_traj.txt

Appending len(dftmp):1113

File opened and loaded : records/MA/fx2/follow_traj.txt

Appending len(dftmp):1185



<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27:

File opened and loaded : records/MA/traj5/follow_traj.txt

Appending len(dftmp):354

File opened and loaded : records/MA/traj9/follow_traj.txt

Appending len(dftmp):360

File opened and loaded : records/MA/fuz1/follow_traj.txt

Appending len(dftmp):1606

File opened and loaded : records/MA/traj8/follow_traj.txt

Appending len(dftmp):362

File opened and loaded : records/MA/fx7/follow_traj.txt

Appending len(dftmp):1190

File opened and loaded : records/MA/fx5/follow_traj.txt

Appending len(dftmp):1008

File opened and loaded : records/MA/fx1/follow_traj.txt

Appending len(dftmp):2517

File opened and loaded : records/MA/fx9/follow_traj.txt

Appending len(dftmp):1182

File opened and loaded : records/move/test_ludo8/follow_traj.txt



<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27:

Appending len(dftmp):3351

File opened and loaded : records/move/test_ludo4/follow_traj.txt

Appending len(dftmp):5736

File filtered not loaded : records/fuzzy/tt/follow_traj.txt

File opened and loaded : records/fuzzy/test_ludo2/follow_traj.txt

Appending len(dftmp):11

File opened and loaded : records/fuzzy/test_ludo/follow_traj.txt

Appending len(dftmp):2808

File filtered not loaded : records/test1/fx/follow_traj.txt

File filtered not loaded : records/test1/av/follow_traj.txt

File filtered not loaded : records/test1/traj/follow_traj.txt

File opened and loaded : records/MMECA/test1/follow_traj.txt

Appending len(dftmp):5666

File opened and loaded : records/MMECA/test2/follow_traj.txt

Appending len(dftmp):4534

File filtered not loaded : records/test2/av/follow_traj.txt

File filtered not loaded : records/test2/traj/follow_traj.txt

File filtered not loaded : records/traj/test/follow_traj.txt

len(df)=98403


<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)
<ipython-input-67-c60b43b4ba2e>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dftmp)


In [82]:
import matplotlib.pyplot as mpl
mydf= [val for val in df if (df['file']=='records/MA/traj1/follow_traj.txt')]
mpl.plot(mydf[0],mydf[1])

ValueError: ignored

In [94]:
[df[0] if (df['file']=='records/MA/traj1/follow_traj.txt')]



SyntaxError: ignored

# Apprentissage des données

## Construction des données de validation et de test

## Apprentissage (scikit)

# Apprentissage (keras)